In [ ]:
#트레이닝, 테스트 데이터 셋 나누기
from sklearn.model_selection import train_test_split

train_test_split()

In [3]:
#의사결정나무
from sklearn.tree import DecisionTreeClassifier

DecisionTreeClassifier(criterion='entropy', max_depth=1, random_state=0).fit(X, y)